In [4]:
import face_recognition
import pickle

# حملي صورة الطفل
child_image = face_recognition.load_image_file(r"C:\projects\sbc\monitor\child_photos\photo_1.jpg")
child_encodings = face_recognition.face_encodings(child_image)

if len(child_encodings) == 0:
    print("❌ لم يتم العثور على وجه في صورة الطفل!")
else:
    with open("child_face_encoding.pkl", "wb") as f:
        pickle.dump(child_encodings[0], f)
    print("✅ تم حفظ encoding وجه الطفل بنجاح!")


✅ تم حفظ encoding وجه الطفل بنجاح!


In [17]:
import cv2
import json

cap = cv2.VideoCapture("video_2025-06-16_12-26-42.mp4")
ret, frame = cap.read()
cap.release()

if not ret:
    print("❌ فشل قراءة أول فريم!")
    exit()

bbox = cv2.selectROI("حدد الكنبة", frame, fromCenter=False, showCrosshair=True)
cv2.destroyAllWindows()

# خزني إحداثيات الكنبة في json
with open("bed_bbox.json", "w") as f:
    json.dump({"x": bbox[0], "y": bbox[1], "w": bbox[2], "h": bbox[3]}, f)

print("✅ تم حفظ حدود الكنبة في bed_bbox.json")


✅ تم حفظ حدود الكنبة في bed_bbox.json


In [1]:
import cv2
import numpy as np
import json
import pickle
import face_recognition
from ultralytics import YOLO
import mediapipe as mp
import time

# ------------------------- تحميل إحداثيات الكنبة من الملف -------------------------
with open("bed_bbox.json", "r") as f:
    bbox_dict = json.load(f)
bbox = (bbox_dict["x"], bbox_dict["y"], bbox_dict["w"], bbox_dict["h"])

# ------------------------- تحميل face encoding للطفل -------------------------
with open("child_face_encoding.pkl", "rb") as f:
    child_encoding = pickle.load(f)

# ------------------------- إعداد موديلات YOLO -------------------------
fall_model = YOLO("runs/detect/train_lowmem3/weights/best.pt")
general_model = YOLO("yolov8m.pt")

# ------------------------- إعداد MediaPipe للكشف عن اليد -------------------------
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# ------------------------- فتح الفيديو -------------------------
cap = cv2.VideoCapture("video_2025-06-16_12-26-42.mp4")
if not cap.isOpened():
    print("❌ تأكد من وجود الفيديو!")
    exit()

# ------------------------- تفعيل التتبع -------------------------
ret, frame = cap.read()
if not ret:
    print("❌ فشل قراءة أول فريم!")
    exit()

tracker = cv2.TrackerCSRT_create()
tracker.init(frame, bbox)

# ------------------------- تشغيل التحليل -------------------------
start_time = time.time()
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # ------------------------ تحديث تتبع الكنبة ------------------------
    ok, bbox = tracker.update(frame)
    if ok:
        (x, y, w, h) = [int(v) for v in bbox]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    else:
        cv2.putText(frame, "❌  monitor faild", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # ------------------------ كشف اليد ------------------------
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            y_wrist, y_tip = hand_landmarks.landmark[0].y, hand_landmarks.landmark[8].y
            if y_tip < y_wrist - 0.1:
                cv2.putText(frame, "🚨 baby hand up !", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # ------------------------ كشف وجه الطفل ------------------------
    face_locations = face_recognition.face_locations(frame_rgb)
    face_encodings = face_recognition.face_encodings(frame_rgb, face_locations)
    found_child = False
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        match = face_recognition.compare_faces([child_encoding], face_encoding, tolerance=0.5)
        if match[0]:
            found_child = True
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, "our baby ✅", (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0),2)
        else:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, "🚨 not baby !", (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255),2)

    if not found_child and face_locations:
        cv2.putText(frame, "🚨 baby not here!", (10, 140), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),2)

    # ------------------------ كشف السقوط ------------------------
    fall_results = fall_model(frame)[0]
    for box in fall_results.boxes:
        conf = float(box.conf[0])
        if conf > 0.5:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, f"Fall Detected! ({conf:.2f})", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255),2)

    # ------------------------ حساب FPS ------------------------
    frame_count += 1
    elapsed_time = time.time() - start_time
    fps = frame_count / elapsed_time
    cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    cv2.imshow("Smart Baby Monitor ❤️", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 416x256 1 Fall-Detected, 144.5ms
Speed: 2.5ms preprocess, 144.5ms inference, 7.2ms postprocess per image at shape (1, 3, 416, 256)

0: 416x256 1 Fall-Detected, 90.1ms
Speed: 0.9ms preprocess, 90.1ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 256)

0: 416x256 3 Fall-Detecteds, 79.3ms
Speed: 1.0ms preprocess, 79.3ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 256)

0: 416x256 1 Fall-Detected, 88.4ms
Speed: 0.8ms preprocess, 88.4ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 256)

0: 416x256 2 Fall-Detecteds, 89.1ms
Speed: 0.8ms preprocess, 89.1ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 256)

0: 416x256 1 Fall-Detected, 83.6ms
Speed: 0.9ms preprocess, 83.6ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 256)

0: 416x256 1 Fall-Detected, 83.9ms
Speed: 0.8ms preprocess, 83.9ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 256)

0: 416x256 3 Fall-Detecteds, 89.7ms
Speed: 0.8ms preprocess, 89.

In [2]:
from ultralytics import YOLO

# حدد موقع ملف data.yaml الخاص بداتاسيت sharp
data_path = r"C:/projects/SBC/monitor/sharp-ite.v1.yolov8/data.yaml"

model = YOLO("yolov8m.pt")  # نستخدم yolo متوسط الحجم

model.train(
    data=data_path,
    epochs=20,
    imgsz=416,
    batch=2,  # مناسب لذاكرة جهازك
    name="sharp_tuned",
    device=0  # ← شغّل التدريب على الـ GPU
)


New https://pypi.org/project/ultralytics/8.3.162 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.114  Python-3.13.3 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=C:/projects/SBC/monitor/sharp-ite.v1.yolov8/data.yaml, epochs=20, time=None, patience=100, batch=2, imgsz=416, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=sharp_tuned2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, 

train: Scanning C:\projects\sbc\monitor\sharp-ite.v1.yolov8\train\labels.cache... 922 images, 1 backgrounds, 0 corrupt: 100%|██████████| 922/922 [00:00<?, ?it/s]

WARNING Box and segment counts should be equal, but got len(segments) = 110, len(boxes) = 1054. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


val: Fast image access  (ping: 0.10.0 ms, read: 98.326.0 MB/s, size: 22.2 KB)


val: Scanning C:\projects\sbc\monitor\sharp-ite.v1.yolov8\valid\labels.cache... 230 images, 0 backgrounds, 0 corrupt: 100%|██████████| 230/230 [00:00<?, ?it/s]

...

                   all        230        303      0.778      0.773      0.869      0.563



20 epochs completed in 0.439 hours.
Optimizer stripped from runs\detect\sharp_tuned2\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\sharp_tuned2\weights\best.pt, 52.0MB

Validating runs\detect\sharp_tuned2\weights\best.pt...
Ultralytics 8.3.114  Python-3.13.3 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 58/58 [00:04<00:00, 13.81it/s]


                   all        230        303      0.767      0.779       0.87      0.564
                 knife        209        282      0.797      0.652      0.796      0.477
              scissors         21         21      0.737      0.905      0.944       0.65
Speed: 0.3ms preprocess, 10.0ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs\detect\sharp_tuned2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000023347775090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [1]:
from ultralytics import YOLO

# حدد موقع ملف data.yaml الخاص بداتاسيت الوضعيات
data_path = r"C:/projects/SBC/monitor/baby-posture.v2i.yolov8/data.yaml"

model = YOLO("yolov8m.pt")  # نفس الموديل الأساسي

model.train(
    data=data_path,
    epochs=30,
    imgsz=416,
    batch=2,
    name="posture_tuned",
    device=0
)


New https://pypi.org/project/ultralytics/8.3.162 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.114  Python-3.13.3 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=C:/projects/SBC/monitor/baby-posture.v2i.yolov8/data.yaml, epochs=30, time=None, patience=100, batch=2, imgsz=416, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=posture_tuned3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=

train: Scanning C:\projects\sbc\monitor\baby-posture.v2i.yolov8\train\labels.cache... 4233 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4233/4233 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 169.918.5 MB/s, size: 49.8 KB)


val: Scanning C:\projects\sbc\monitor\baby-posture.v2i.yolov8\valid\labels.cache... 58 images, 0 backgrounds, 0 corrupt: 100%|██████████| 58/58 [00:00<?, ?it/s]


Plotting labels to runs\detect\posture_tuned3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 8 dataloader workers
Logging results to runs\detect\posture_tuned3
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         58         58       0.02      0.834     0.0244    0.00646



30 epochs completed in 2.699 hours.
Optimizer stripped from runs\detect\posture_tuned3\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\posture_tuned3\weights\best.pt, 52.0MB

Validating runs\detect\posture_tuned3\weights\best.pt...
Ultralytics 8.3.114  Python-3.13.3 torch-2.7.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:01<00:00, 13.20it/s]


                   all         58         58      0.303      0.653       0.33      0.193
          baby_on_back         17         17      0.207      0.647      0.237      0.109
       baby_on_stomach         41         41      0.399      0.659      0.423      0.277
Speed: 0.3ms preprocess, 11.9ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs\detect\posture_tuned3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001636561CEC0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0